## Practica 1 - PIAV

Integrantes: Alejandro Bolaños García y David García Díaz

##### 1a. Hacer un programa que cargue una imagen y que obtenga y muestre el valor RGB de cada píxel de la imagen sobre el que esté el cursor del ratón.


In [22]:
import cv2 as cv
import numpy as np

# Función de callback para el ratón
def mouse(event, x, y, flags, param):
    if event == cv.EVENT_MOUSEMOVE:  # Detectar el movimiento del ratón
        # Obtener el valor RGB del píxel en la posición (x, y)
        b, g, r = img[y, x]  # Los valores se obtienen en orden BGR en OpenCV
        #print(f'Posición: ({x}, {y}), Valor RGB: ({r}, {g}, {b})')
        
        # Mostrar el valor RGB en la imagen
        img_copy = img.copy()
        text = f'RGB: ({r}, {g}, {b})'
        cv.putText(img_copy, text, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv.LINE_AA)
        cv.imshow('imagen', img_copy)

# Cargar la imagen
img = cv.imread('images/bosque.jpg')

# Mostrar la imagen
cv.imshow('imagen', img)

# Configurar el callback del ratón
cv.setMouseCallback('imagen', mouse)

cv.waitKey(0)
cv.destroyAllWindows()

##### 1b. Hacer un programa para dibujar las siguientes primitivas con el ratón:
- Líneas
- Rectángulo
- Círculos
- Especificar el color en RGB de las primitivas
- Definir el grosor de las líneas

##### 2. Mejoras del programa
- Permitir crear figuras rellenas (seleccionar a través de la interfaz.
- Poder crear otras primitivas (construir polilíneas y polígonos, elipses…)
- Guardar el dibujo como un vídeo para ver cómo se realizó paso a paso
- Aportación propia

In [7]:
# SIN BOTON DE RELLENO

import numpy as np
import cv2 as cv

drawing = False  # Verdadero si el ratón está presionado
mode = 0  # 0: rectángulo, 1: círculo, 2: línea, 3: polilínea, 4: polígono, 5: elipse
ix, iy = -1, -1
points = []  # Lista para almacenar puntos de polilíneas y polígonos
video_writer = None
fill = False  # Variable para el estado de relleno

# Lista de nombres de modos para la leyenda
mode_names = ['Rectangulo', 'Circulo', 'Linea', 'Polilinea', 'Poligono', 'Elipse']

def trackbar(x):
    pass

# Función para inicializar el video writer
def initialize_video_writer():
    global video_writer
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    video_writer = cv.VideoWriter('drawing.avi', fourcc, 2.0, (1000, 1000))

# Función de callback para el ratón
def draw_figures(event, x, y, flags, param):
    global ix, iy, drawing, mode, points, fill

    if event == cv.EVENT_LBUTTONDOWN:
        # Verificar si se ha hecho clic en el botón de relleno
        if 10 <= x <= 110 and 10 <= y <= 50:
            fill = not fill
            return

        # Solo empezar a dibujar si no se ha hecho clic en el botón
        drawing = True
        ix, iy = x, y

        # Para polilínea o polígono, añadir punto y dibujar
        if mode == 3 or mode == 4:  # Polilínea o Polígono
            points.append((x, y))  # Añadir punto a la lista

    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()  # Copia de la imagen para dibujar en tiempo real
            thickness = cv.getTrackbarPos('Thickness', 'image')
            color = (cv.getTrackbarPos('B', 'image'), cv.getTrackbarPos('G', 'image'), cv.getTrackbarPos('R', 'image'))

            # Verificar si se desea rellenar la figura
            if fill:
                thickness = -1  # Rellenar la figura

            # Dibujar según el modo
            if mode == 0:  # Rectángulo
                cv.rectangle(img_copy, (ix, iy), (x, y), color, thickness)
            elif mode == 1:  # Círculo
                radius = int(np.sqrt((x - ix) ** 2 + (y - iy) ** 2))
                cv.circle(img_copy, (ix, iy), radius, color, thickness)
            elif mode == 2:  # Línea
                cv.line(img_copy, (ix, iy), (x, y), color, cv.getTrackbarPos('Thickness', 'image'))
            elif mode == 5:  # Elipse
                cv.ellipse(img_copy, (ix, iy), (abs(x - ix), abs(y - iy)), 0, 0, 360, color, thickness)
            elif mode == 3 or mode == 4:  # Polilínea o Polígono
                # Dibujar los puntos de la polilínea/polígono en tiempo real
                if len(points) > 1:
                    if fill and mode == 4:
                        # Rellenar el polígono
                        cv.fillPoly(img_copy, [np.array(points, np.int32)], color)
                    else:
                        cv.polylines(img_copy, [np.array(points, np.int32)], mode == 4, color, thickness)

            # Añadir controles y leyenda
            draw_controls(img_copy)
            cv.imshow('image', img_copy)

    elif event == cv.EVENT_RBUTTONDOWN:
        # Completa el polígono o la polilínea al hacer clic derecho
        if mode == 3 or mode == 4:
            draw_polyline_or_polygon(finalize=True)
            points.clear()  # Limpiar puntos después de completar el dibujo

    elif event == cv.EVENT_LBUTTONUP:
        # Si no estamos dibujando, no hacer nada
        if not drawing:
            return

        drawing = False
        thickness = cv.getTrackbarPos('Thickness', 'image')
        color = (cv.getTrackbarPos('B', 'image'), cv.getTrackbarPos('G', 'image'), cv.getTrackbarPos('R', 'image'))

        # Verificar si se desea rellenar la figura
        if fill:
            thickness = -1  # Rellenar la figura

        # Dibujar según el modo
        if mode == 0:  # Rectángulo
            cv.rectangle(img, (ix, iy), (x, y), color, thickness)
        elif mode == 1:  # Círculo
            radius = int(np.sqrt((x - ix) ** 2 + (y - iy) ** 2))
            cv.circle(img, (ix, iy), radius, color, thickness)
        elif mode == 2:  # Línea
            cv.line(img, (ix, iy), (x, y), color, cv.getTrackbarPos('Thickness', 'image'))
        elif mode == 5:  # Elipse
            cv.ellipse(img, (ix, iy), (abs(x - ix), abs(y - iy)), 0, 0, 360, color, thickness)

        # Guardar el frame en el video
        if video_writer:
            video_writer.write(img)

def draw_polyline_or_polygon(finalize=False):
    thickness = cv.getTrackbarPos('Thickness', 'image')
    color = (cv.getTrackbarPos('B', 'image'), cv.getTrackbarPos('G', 'image'), cv.getTrackbarPos('R', 'image'))
    if fill:
        thickness = -1  # Rellenar la figura

    # Dibujar polilínea o polígono en la imagen principal
    if mode == 3:  # Polilínea
        cv.polylines(img, [np.array(points, np.int32)], False, color, thickness)
    elif mode == 4:  # Polígono
        if finalize:
            if fill:
                # Rellenar el polígono
                cv.fillPoly(img, [np.array(points, np.int32)], color)
            else:
                # Dibujar solo el contorno
                cv.polylines(img, [np.array(points, np.int32)], True, color, thickness)

def draw_controls(image):
    # Dibujar el botón de relleno
    button_text = 'Relleno: ON' if fill else 'Relleno: OFF'
    button_color = (0, 255, 0) if fill else (0, 0, 255)  # Verde si está en ON, rojo si está en OFF
    
    cv.rectangle(image, (10, 10), (110, 50), button_color, -1)  # Fondo del botón
    cv.putText(image, button_text, (15, 35), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv.LINE_AA)

    # Añadir la leyenda del modo actual
    cv.putText(image, f'Modo: {mode_names[mode]}', (10, 70), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv.LINE_AA)

# Configuración inicial
img = np.zeros((1000, 1000, 3), np.uint8)
cv.namedWindow('image')
cv.setMouseCallback('image', draw_figures)

# Crear los trackbars
cv.createTrackbar('R', 'image', 255, 255, trackbar)
cv.createTrackbar('G', 'image', 255, 255, trackbar)
cv.createTrackbar('B', 'image', 255, 255, trackbar)
cv.createTrackbar('Thickness', 'image', 1, 10, trackbar)  # Inicia en 1 para evitar grosor 0

initialize_video_writer()  # Inicializar el video writer

while True:
    # Mostrar la imagen con los controles
    img_copy = img.copy()
    draw_controls(img_copy)
    cv.imshow('image', img_copy)

    k = cv.waitKey(1) & 0xFF
    if k == ord('c'):
        mode = 1  # Cambia a modo círculo
    elif k == ord('l'):
        mode = 2  # Cambia a modo línea
    elif k == ord('r'):
        mode = 0  # Cambia a modo rectángulo
    elif k == ord('p'):
        mode = 3  # Cambia a modo polilínea
    elif k == ord('o'):
        mode = 4  # Cambia a modo polígono
    elif k == ord('s'):  # Guardar la imagen actual
        cv.imwrite('drawing.png', img)
    elif k == ord('n'):  # Limpiar la imagen
        img = np.zeros((1000, 1000, 3), np.uint8)
        points = []  # Limpiar los puntos para las polilíneas y polígonos
    elif k == 27:  # Tecla 'ESC' para salir
        break

video_writer.release()  # Liberar el video writer
cv.destroyAllWindows()


In [6]:
# SIN BOTON DE RELLENO

import numpy as np
import cv2 as cv

drawing = False  # Verdadero si el ratón está presionado
mode = 0  # 0: rectángulo, 1: círculo, 2: línea, 3: polilínea, 4: polígono, 5: elipse
ix, iy = -1, -1
points = []  # Lista para almacenar puntos de polilíneas y polígonos
video_writer = None

# Lista de nombres de modos para la leyenda
mode_names = ['Rectangulo', 'Circulo', 'Linea', 'Polilinea', 'Poligono', 'Elipse']

def trackbar(x):
    pass

# Función para inicializar el video writer
def initialize_video_writer():
    global video_writer
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    video_writer = cv.VideoWriter('drawing.avi', fourcc, 2.0, (1000, 1000))

# Función de callback para el ratón
def draw_figures(event, x, y, flags, param):
    global ix, iy, drawing, mode, points

    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        if mode == 3 or mode == 4:  # Polilínea o Polígono
            points.append((x, y))  # Añadir punto a la lista

        # Para polilínea o polígono, añadir punto y dibujar
        if mode == 3 or mode == 4:  # Polilínea o Polígono
            points.append((x, y))  # Añadir punto a la lista

    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()  # Copia de la imagen para dibujar en tiempo real
            thickness = cv.getTrackbarPos('Thickness', 'image')
            color = (cv.getTrackbarPos('B', 'image'), cv.getTrackbarPos('G', 'image'), cv.getTrackbarPos('R', 'image'))

            # Verificar si se desea rellenar la figura
            if cv.getTrackbarPos('Relleno', 'image') == 1:
                thickness = -1  # Rellenar la figura

            # Dibujar según el modo
            if mode == 0:  # Rectángulo
                cv.rectangle(img_copy, (ix, iy), (x, y), color, thickness)
            elif mode == 1:  # Círculo
                radius = int(np.sqrt((x - ix) ** 2 + (y - iy) ** 2))
                cv.circle(img_copy, (ix, iy), radius, color, thickness)
            elif mode == 2:  # Línea
                cv.line(img_copy, (ix, iy), (x, y), color, cv.getTrackbarPos('Thickness', 'image'))
            elif mode == 5:  # Elipse
                cv.ellipse(img_copy, (ix, iy), (abs(x - ix), abs(y - iy)), 0, 0, 360, color, thickness)
            elif mode == 3 or mode == 4:  # Polilínea o Polígono
                # Dibujar los puntos de la polilínea/polígono en tiempo real
                if len(points) > 1:
                    if thickness==-1 and mode == 4:
                        # Rellenar el polígono
                        cv.fillPoly(img_copy, [np.array(points, np.int32)], color)
                    else:
                        cv.polylines(img_copy, [np.array(points, np.int32)], mode == 4, color, thickness)

            # Añadir la leyenda del modo actual
            cv.putText(img_copy, f'Modo: {mode_names[mode]}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)

            cv.imshow('image', img_copy)
    
    elif event == cv.EVENT_RBUTTONDOWN:
        # Completa el polígono o la polilínea al hacer clic derecho
        if mode == 3 or mode == 4:
            thickness = cv.getTrackbarPos('Thickness', 'image')
        color = (cv.getTrackbarPos('B', 'image'), cv.getTrackbarPos('G', 'image'), cv.getTrackbarPos('R', 'image'))
        if cv.getTrackbarPos('Relleno', 'image') == 1:
            thickness = -1  # Rellenar la figura

        # Dibujar polilínea o polígono en la imagen principal
        if mode == 3:  # Polilínea
            cv.polylines(img, [np.array(points, np.int32)], False, color, thickness)
        elif mode == 4:  # Polígono
            if cv.getTrackbarPos('Relleno', 'image') == 1:
                # Rellenar el polígono
                cv.fillPoly(img, [np.array(points, np.int32)], color)
            else:
                # Dibujar solo el contorno
                cv.polylines(img, [np.array(points, np.int32)], True, color, thickness)

            points.clear()  # Limpiar puntos después de completar el dibujo

    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        thickness = cv.getTrackbarPos('Thickness', 'image')
        color = (cv.getTrackbarPos('B', 'image'), cv.getTrackbarPos('G', 'image'), cv.getTrackbarPos('R', 'image'))

        # Verificar si se desea rellenar la figura
        if cv.getTrackbarPos('Relleno', 'image') == 1:
            thickness = -1  # Rellenar la figura

        # Dibujar según el modo
        if mode == 0:  # Rectángulo
            cv.rectangle(img, (ix, iy), (x, y), color, thickness)
        elif mode == 1:  # Círculo
            radius = int(np.sqrt((x - ix) ** 2 + (y - iy) ** 2))
            cv.circle(img, (ix, iy), radius, color, thickness)
        elif mode == 2:  # Línea
            cv.line(img, (ix, iy), (x, y), color, cv.getTrackbarPos('Thickness', 'image'))
        elif mode == 5:  # Elipse
            cv.ellipse(img, (ix, iy), (abs(x - ix), abs(y - iy)), 0, 0, 360, color, thickness)

        # Guardar el frame en el video
        if video_writer:
            video_writer.write(img)

# Configuración inicial
img = np.zeros((1000, 1000, 3), np.uint8)
cv.namedWindow('image')
cv.setMouseCallback('image', draw_figures)

# Crear los trackbars
cv.createTrackbar('Relleno', 'image', 0, 1, trackbar)
cv.createTrackbar('R', 'image', 255, 255, trackbar)
cv.createTrackbar('G', 'image', 0, 255, trackbar)
cv.createTrackbar('B', 'image', 0, 255, trackbar)
cv.createTrackbar('Thickness', 'image', 1, 10, trackbar)  # Inicia en 1 para evitar grosor 0

initialize_video_writer()  # Inicializar el video writer

while True:
    # Mostrar el modo actual en la imagen
    img_copy = img.copy()
    cv.putText(img_copy, f'Modo: {mode_names[mode]}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    cv.imshow('image', img_copy)

    k = cv.waitKey(1) & 0xFF
    if k == ord('c'):
        mode = 1  # Cambia a modo círculo
    elif k == ord('l'):
        mode = 2  # Cambia a modo línea
    elif k == ord('r'):
        mode = 0  # Cambia a modo rectángulo
    elif k == ord('p'):
        mode = 3  # Cambia a modo polilínea
    elif k == ord('o'):
        mode = 4  # Cambia a modo polígono
    elif k == ord('e'):
        mode = 5  # Cambia a modo elipse
    elif k == ord('s'):  # Guardar la imagen actual
        cv.imwrite('drawing.png', img)
    elif k == ord('n'):  # Limpiar la imagen
        img = np.zeros((1000, 1000, 3), np.uint8)
        points = []  # Limpiar los puntos para las polilíneas y polígonos
    elif k == 27:  # Tecla 'ESC' para salir
        break

video_writer.release()  # Liberar el video writer
cv.destroyAllWindows()